In [1]:
import pandas as pd

# 1. 定义列名 (Criteo数据集没有表头，我们需要自己加上)
# Label: 0或1，表示是否被点击
# I1-I13: 13个整数特征 (Integer Features)
# C1-C26: 26个类别特征 (Categorical Features)
cols = ['label'] + [f'I{i}' for i in range(1, 14)] + [f'C{i}' for i in range(1, 27)]

# 2. 设置文件路径
file_path = 'D:/PythonProjects/recommendation/kaggle-CTR/kaggle-display-advertising-challenge-dataset/train.txt' 

# 3. 读取前 10 万行看看长什么样
# sep='\t' 表示是用 Tab 键分隔的
try:
    df = pd.read_csv(file_path, sep='\t', names=cols, nrows=2000000)
    print("数据读取成功！形状为:", df.shape)

    # 展示前 5 行
    display(df.head())

    # 看看每一列的数据类型
    print("\n数据类型概览:")
    print(df.info())

except FileNotFoundError:
    print("找不到文件，请检查 file_path 是否写对了。")
except Exception as e:
    print("出错了:", e)

数据读取成功！形状为: (2000000, 40)


,label,I1,I2,I3,I4,I5,I6,I7,I8,I9,...,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26
0,0,1.0,1,5.0,0.0,1382.0,4.0,15.0,2.0,181.0,...,e5ba7672,f54016b9,21ddcdc9,b1252a9d,07b5194c,NaN,3a171ecb,c5c50484,e8b83407,9727dd16
1,0,2.0,0,44.0,1.0,102.0,8.0,2.0,2.0,4.0,...,07c540c4,b04e4670,21ddcdc9,5840adea,60f6221e,NaN,3a171ecb,43f13e8b,e8b83407,731c3655
2,0,2.0,0,1.0,14.0,767.0,89.0,4.0,2.0,245.0,...,8efede7f,3412118d,NaN,NaN,e587c466,ad3062eb,3a171ecb,3b183c5c,NaN,NaN
3,0,NaN,893,NaN,NaN,4392.0,NaN,0.0,0.0,0.0,...,1e88c74f,74ef3502,NaN,NaN,6b3a5ca6,NaN,3a171ecb,9117a34a,NaN,NaN
4,0,3.0,-1,NaN,0.0,2.0,0.0,3.0,0.0,0.0,...,1e88c74f,26b3c7a7,NaN,NaN,21c9516a,NaN,32c7478e,b34f3128,NaN,NaN



数据类型概览:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000000 entries, 0 to 1999999
Data columns (total 40 columns):
 #   Column  Dtype  
---  ------  -----  
 0   label   int64  
 1   I1      float64
 2   I2      int64  
 3   I3      float64
 4   I4      float64
 5   I5      float64
 6   I6      float64
 7   I7      float64
 8   I8      float64
 9   I9      float64
 10  I10     float64
 11  I11     float64
 12  I12     float64
 13  I13     float64
 14  C1      object 
 15  C2      object 
 16  C3      object 
 17  C4      object 
 18  C5      object 
 19  C6      object 
 20  C7      object 
 21  C8      object 
 22  C9      object 
 23  C10     object 
 24  C11     object 
 25  C12     object 
 26  C13     object 
 27  C14     object 
 28  C15     object 
 29  C16     object 
 30  C17     object 
 31  C18     object 
 32  C19     object 
 33  C20     object 
 34  C21     object 
 35  C22     object 
 36  C23     object 
 37  C24     object 
 38  C25     object 
 39  C26     obje

In [2]:
print(df.isnull().sum())

label          0
I1        849735
I2             0
I3        454348
I4        473222
I5         45417
I6        437836
I7         79899
I8          1197
I9         79899
I10       849735
I11        79899
I12      1542352
I13       473222
C1             0
C2             0
C3         69482
C4         69482
C5             0
C6        223355
C7             0
C8             0
C9             0
C10            0
C11            0
C12        69482
C13            0
C14            0
C15            0
C16        69482
C17            0
C18            0
C19       941343
C20       941343
C21        69482
C22      1465691
C23            0
C24        69482
C25       941343
C26       941343
dtype: int64


In [3]:
from sklearn.preprocessing import StandardScaler
import numpy as np

# 1. 定义数值特征列名 
dense_features = [f'I{i}' for i in range(1, 14)]
sparse_features = [f'C{i}' for i in range(1, 27)]

# 2. 处理数值特征 (I1-I13)
# 第一步：空值填 0
df[dense_features] = df[dense_features].fillna(0)

########################################################################################################
# ===  关键修改：用 StandardScaler 代替 Log 变换 ===
# 这一步会把数值特征变成 0 附近的小数 (比如 -1.2, 0.5, 2.1)
# 这样模型就不容易梯度爆炸了 
scaler = StandardScaler()
df[dense_features] = scaler.fit_transform(df[dense_features])

# 3. 处理类别特征 (C1-C26): 空值填 "-1"
df[sparse_features] = df[sparse_features].fillna('-1')

# 4. 再次检查
print("数值特征标准化完成！")
print("剩余缺失值数量:", df.isnull().sum().sum())

数值特征标准化完成！
剩余缺失值数量: 0


In [4]:
from sklearn.preprocessing import LabelEncoder

# 这里的字典用来保存每一列的 Encoder，以防以后预测时要用
le_dict = {}

for col in [f'C{i}' for i in range(1, 27)]:
    le = LabelEncoder()
    # fit_transform 会把这一列的字符串变成 0, 1, 2... 的数字
    df[col] = le.fit_transform(df[col])
    le_dict[col] = le

print("编码完成，来看看现在的样子:")
display(df.head)

编码完成，来看看现在的样子:


<bound method NDFrame.head of          label        I1        I2        I3        I4        I5        I6  \
0            0 -0.126108 -0.268473 -0.042678 -0.667604 -0.249399 -0.239015   
1            0  0.016668 -0.271109  0.111554 -0.541726 -0.268222 -0.227721   
2            0  0.016668 -0.271109 -0.058497  1.094679 -0.258443  0.000997   
3            0 -0.268884  2.082906 -0.062451 -0.667604 -0.205136 -0.250310   
4            0  0.159443 -0.273745 -0.062451 -0.667604 -0.269692 -0.250310   
...        ...       ...       ...       ...       ...       ...       ...   
1999995      1 -0.268884  1.002115 -0.062451 -0.415849 -0.037790 -0.128892   
1999996      0  0.159443 -0.268473 -0.050587 -0.667604 -0.256604 -0.241839   
1999997      1 -0.126108 -0.273745 -0.062451 -0.667604 -0.248590 -0.247486   
1999998      1  1.444426 -0.271109  0.072007  0.842925 -0.268824 -0.213602   
1999999      0 -0.268884 -0.271109 -0.054542 -0.415849 -0.227532 -0.148658   

               I7        I8      

In [5]:
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import numpy as np

# 1. 定义列名
dense_features = [f'I{i}' for i in range(1, 14)]
sparse_features =[f'C{i}' for i in range(1, 27)]

# 2. 提取数据 (转换为 numpy 数组)
# label: 目标值
# dense_x: 数值特征矩阵
# sparse_x: 类别特征矩阵
labels = df['label'].values
dense_x = df[dense_features].values
sparse_x = df[sparse_features].values

# 3. 切分训练集和验证集 (8:2)
print("正在切分数据...")
train_dense, val_dense, train_sparse, val_sparse, train_label, val_label =train_test_split(
    dense_x, sparse_x, labels, test_size = 0.2, random_state = 2025
)

print(f"训练集数量：{len(train_label)}")
print(f"验证机数量：{len(val_label)}")

正在切分数据...
训练集数量：1600000
验证机数量：400000


In [6]:
class CriteoDataset(Dataset):
    def __init__(self, dens_x, sparse_x, label):
        """
        初始化函数：把 numpy 数组转成 tensor
        """
        # 数值特征 -> float32
        self.dense_x = torch.tensor(dense_x, dtype = torch.float32)
        # 类别特征 -> long (必须是整数，否则 Embedding 层会报错)
        self.sparse_x = torch.tensor(sparse_x, dtype = torch.long)
        # 标签 -> float32 (为了配合 BCE Loss 计算)
        self.label = torch.tensor(label, dtype = torch.float32)

    def __len__(self):
        # 告诉 DataLoader 数据集有多长
        return len(self.label)

    def __getitem__(self, idx):
        # 告诉 DataLoader 怎么取第 idx 条数据
        return self.dense_x[idx], self.sparse_x[idx], self.label[idx]

In [7]:
# 1. 实例化 Dataset
train_dataset = CriteoDataset(train_dense, train_sparse, train_label)
val_dataset = CriteoDataset(val_dense, val_sparse, val_label)

# 2. 实例化 DataLoader
BATCH_SIZE = 512

train_loader = DataLoader(train_dataset, batch_size = BATCH_SIZE, shuffle = True, num_workers = 0)
val_loader = DataLoader(val_dataset, batch_size = BATCH_SIZE, shuffle = False, num_workers = 0)

# === ✅ 验证环节  ===
# 从管道里取一个 Batch 出来看看形状
dense_batch, sparse_batch, label_batch = next(iter(train_loader))

print("\n=== 管道测试报告 ===")
print(f"Dense Batch Shape: {dense_batch.shape}  (预期[2048, 13])")
print(f"Sparse Batch Shape: {sparse_batch.shape}(预期[2048, 26])")
print(f"Label Batch Shape: {label_batch.shape}  (预期[2048])")

if dense_batch.shape == (BATCH_SIZE, 13) and sparse_batch.dtype == torch.int64:
    print("√测试通过！数据管道搭建成功。")
else:
    print("× 测试失败，请检查形状或数据类型。")
    


=== 管道测试报告 ===
Dense Batch Shape: torch.Size([512, 13])  (预期[2048, 13])
Sparse Batch Shape: torch.Size([512, 26])(预期[2048, 26])
Label Batch Shape: torch.Size([512])  (预期[2048])
√测试通过！数据管道搭建成功。


In [8]:
# 1. 获取类别特征的词表大小 (Vocabulary Size)
# 也就是每一列有多少个不同的整数索引
sparse_feat_sizes = [df[f'C{i}'].nunique() for i in range(1, 27)]

# 2. 获取数值特征的数量
dense_feat_num = len(dense_features)

print(f"类别特征词表大小示例：{sparse_feat_sizes[:5]} ...")
print(f"数值特征的数量：{dense_feat_num}")


类别特征词表大小示例：[1370, 541, 597026, 200785, 284] ...
数值特征的数量：13


In [9]:
import torch.nn as nn
import torch.nn.functional as F

class DeepFM(nn.Module):
    def __init__(self, sparse_feat_sizes, dense_feat_num, embedding_dim = 8, hidden_units = [256, 128, 64], dropout = 0.5):
        """
        Args:
            sparse_feat_sizes: 一个列表，包含26个类别特征各自的词表大小
            dense_feat_num: 数值特征的数量 (13)
            embedding_dim: Embedding 向量的长度 (k)，论文通常设为 8 或 10
            hidden_units: DNN 的层数和每层神经元数量
            dropout: 防止过拟合的概率
        """
        super(DeepFM, self).__init__()
        self.dense_feat_num = dense_feat_num
        self.sparse_feat_num = len(sparse_feat_sizes)
        # === 1. Embedding 层 (Shared Input) ===
            # 为每个类别特征建立一个 Embedding 表
            # ModuleList 就像一个列表，里面存了 26 个 nn.Embedding
        self.embeddings = nn.ModuleList([nn.Embedding(vocab_size, embedding_dim) for vocab_size in sparse_feat_sizes])
        # === 2. FM 部分 (First Order & Second Order) ===
            # 一阶 (Linear) 部分：
            # 类别特征的一阶权重：本质上就是 Embedding_dim=1 的 Embedding
        self.fm_1st_order_sparse = nn.ModuleList([nn.Embedding(vocab_size, 1) for vocab_size in sparse_feat_sizes
                                             ])
        # 数值特征的一阶权重：全连接层 13 -> 1
        self.fm_1st_order_dense = nn.Linear(dense_feat_num, 1)

        # 二阶 (Interaction) 部分：不需要额外的参数，直接用上面的 self.embeddings 计算

        # === 3. DNN 部分 (Deep Component) ===
            # 计算 DNN 的输入维度：
            # 输入 = (类别特征数 * Embedding维度) + 数值特征数
            # 比如：26 * 8 + 13 = 221
        self.input_dim = self.sparse_feat_num * embedding_dim + dense_feat_num

        layers = []
        input_dim = self.input_dim
        # 循环搭建全连接层 (Linear -> ReLU -> Dropout)
        for unit in hidden_units:
            layers.append(nn.Linear(input_dim, unit))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout))
            input_dim = unit
        # 最后一层：映射到 1 个输出
        layers.append(nn.Linear(input_dim, 1))

        self.dnn = nn.Sequential(*layers)

    def forward(self, dense_x, sparse_x):
        """
        dense_x: [batch_size, 13]
        sparse_x: [batch_size, 26]
        """
        # === A. Embedding 处理 ===
        # 把 26 个 long 类型的索引变成 26 个向量
        # embedded_sparse_features 形状: [batch, 26, embedding_dim]
        embeds = [emb(sparse_x[:, i]) for i, emb in enumerate(self.embeddings)]
        '''之前没分清楚的一点是nn.Embedding和self.embeddings是不一样的，
        self.embeddings是有着26个nn.Embedding的列表，
        i取的就是self.embeddings对应的第i类的也就是Ci类对应的nn.Embedding。
        而这个nn.Embedding有着N行样本，每个样本对应着隐向量。
        这里相当于是i就是第i行或者self.embeddings的Ci类，
        而emb实际上就是第i个nn.Embedding。
        粗俗点理解就是i是self.embeddings的索引或者自变量，
        而emb是因变量也就是对应索引对应的这个列表中的值。
        而sparse_x本身存的就是这个batch里的id，
        也就是nn.Embedding的索引或者“钥匙”。
        这时候把一整行的sparse_x拉过来解锁，
        第i列也就是第i类的全部在sparse_x里的id都用到了。
        现在还没有具体的值，后面在训练过程中nn.Embedding里面的各类拥有的id会变。
        emb不是什么列表元组，它就是对应的nn.Embedding，一个实例
        而sparse_x就是数据集.那nn.embedding是什么类的，我知道他是一个矩阵等价于emb，不管那么多了，
        反正就相当于提取sparse_x中的号码，可以在这个N*K的矩阵中取走号码对应的行
        总结： DeepFM 通过 nn.Embedding 这一层，把“没法算”的 1 亿维稀疏 ID，变成了“好算”的 8 维稠密向量。
        而且这个 8 维向量同时承载了 FM 的二阶交叉任务和 DNN 的高阶拟合任务，实现了**端到端（End-to-End）**的训练，
        不再需要人工去组合特征 。
        
        '''
        embeds = torch.stack(embeds, dim = 1) 
        '''embeds (张量) = [B, 26, K],在原本的第dim维度的地方插入一个新的维度，
        沿着这个维度堆叠原来的26个独立张量。相当于新开一个轴，或者说类似于26张豆腐皮在一根烤串上
        新的坐标轴（维度）可以索引这些豆腐皮。[样本, 特征, 向量] 的标准矩阵形式。'''

        # === B. FM 一阶部分 (Linear) ===
        # 1. 类别特征查表 (dim=1) -> 求和，这里用cat在原本的"特征"这一维度上concatenate连接而不同于上面的stack新开一个维度堆叠
        fm_1st_sparse_res = [emb(sparse_x[:, i]) for i, emb in enumerate(self.fm_1st_order_sparse)]
        fm_1st_sparse_res = torch.cat(fm_1st_sparse_res, dim = 1)
        fm_1st_sparse_sum = torch.sum(fm_1st_sparse_res, dim = 1, keepdim = True) # [batch, 1] True保持这个维度不被消灭，维持张量形状

        # 2. 数值特征 Linear -> [batch, 1]
        fm_1st_dense_res = self.fm_1st_order_dense(dense_x)

        # 3. 合并
        fm_1st_part = fm_1st_sparse_sum + fm_1st_dense_res

        # === C. FM 二阶部分 (Interaction) ===
        # 公式：0.5 * sum( (sum(vx))^2 - sum(v^2 * x^2) )
        ###把两个隐向量的点积用数学手段变成这个公式，缩小计算量，其实就是高中的期望的公式的变体
        # 这里 x=1 (因为我们只对类别特征做二阶交叉，且隐式 x=1) 数值特征没必要，简化了，直接用DNN就可以有交互

        # 1. 和的平方
        sum_square = torch.pow(torch.sum(embeds, dim = 1), 2)
        # 2. 平方的和
        square_sum = torch.sum(torch.pow(embeds, 2), dim = 1)
        # 3. 相减除以2
        fm_2nd_part = 0.5 * torch.sum(sum_square - square_sum, dim = 1, keepdim = True)

        # === D. DNN 部分 ===
        # 1. 把 Embedding 展平: [batch, 26 * 8]
        #通过展平，我们成功地将 $26$ 个特征向量首尾相接，创建了一个包含所有嵌入信息的长向量（维度 $208$），这构成了 DNN 输入的稀疏特征部分。
        batch_size = sparse_x.shape[0]
        '''因为 sparse_x.shape 返回的是一个元组 (2048, 26)，
        要获取元组中的第一个元素（即 $0$ 索引处的元素），
        我们必须使用 中括号 [] 进行索引。[0] 代表获取元组中的第一个值，即 Batch Size。'''
        flatten_embeds = embeds.view(batch_size, -1)
        
        
        # 2. 拼接数值特征: [batch, 26*8 + 13] 沿着特征维度
        dnn_input = torch.cat([flatten_embeds, dense_x], dim = 1)

        # 3. 跑 MLP
        dnn_output = self.dnn(dnn_input)

        # === E. 最终结果 (Sum) ===
        # DeepFM = FM一阶 + FM二阶 + DNN输出
        total_logit = fm_1st_part + fm_2nd_part + dnn_output

        # 这里的输出是 Logit (还没有过 Sigmoid)，因为我们后面会用 BCEWithLogitsLoss
        return total_logit

print("DeepFM 模型定义完成！")

        

DeepFM 模型定义完成！


In [10]:
import torch.nn as nn
###############################################################################
# ===  1. 定义初始化权重函数 ===
def init_weights(m):
    # 如果是全连接层 (Linear) -> 用 Xavier 初始化
    if isinstance(m, nn.Linear):
        nn.init.xavier_normal_(m.weight)
        if m.bias is not None:
            nn.init.constant_(m.bias, 0)
    # 如果是 Embedding 层 -> 用方差极小的正态分布初始化 (防止梯度爆炸的关键!)
    elif isinstance(m, nn.Embedding):
        nn.init.normal_(m.weight, mean=0, std=0.01)

# === 2. 准备设备 ===
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# === 3. 实例化模型 ===
# 注意：hidden_units 里的参数名 hidden_units 要和你的 DeepFM 类定义一致(有没有s)
model = DeepFM(sparse_feat_sizes, dense_feat_num, embedding_dim=8, hidden_units=[256, 128, 64]).to(device)

# ===  4. 应用初始化  ===
model.apply(init_weights)
print("模型参数已按 Xavier 和 Normal(0.01) 重新初始化！")

# === 5. 测试 Forward ===
# 注意：要把数据也搬到 GPU 上
# 这里的 dense_batch 和 sparse_batch 来自你上一个 DataLoader Cell 的测试结果
dense_batch = dense_batch.to(device)
sparse_batch = sparse_batch.to(device)

# 调用模型
output = model(dense_batch, sparse_batch) 

# 自动获取当前的 Batch Size (防止你把 2048 改成 512 后这里报错)
current_batch_size = dense_batch.shape[0]

print(f"模型输出形状：{output.shape} (预期：[{current_batch_size}, 1])")

if output.shape == (current_batch_size, 1):
    print("√ 模型搭建成功！前向传播没问题。")
else:
    print("× 模型输出形状不对，请检查代码。")

Using device: cuda
模型参数已按 Xavier 和 Normal(0.01) 重新初始化！
模型输出形状：torch.Size([512, 1]) (预期：[512, 1])
√ 模型搭建成功！前向传播没问题。


In [11]:
###训练循环###
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import roc_auc_score, log_loss
import time

# === 1. 准备工作 (Setup) ===
# 检查是否有 GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 实例化模型
# 注意：sparse_feat_sizes 和 dense_feat_num 必须是你之前数据处理步骤算出来的变量
model = DeepFM(sparse_feat_sizes, dense_feat_num, embedding_dim = 8, hidden_units = [256, 128, 64]).to(device)

# 定义损失函数: 二分类交叉熵 (BCEWithLogitsLoss 自带 Sigmoid，数值稳定性更好)
criterion = nn.BCEWithLogitsLoss()

# 定义优化器: Adam (通常是推荐系统的首选)
optimizer = optim.Adam(model.parameters(), lr = 0.001, weight_decay = 1e-5) #告诉优化器所有可学习的参数以及设定学习率

# === 2. 定义训练函数 (Train Engine) ===
def train_one_epoch(model, train_loader, epoch_idx):
    model.train() # 切换到训练模式 (启用 Dropout)
    total_loss = 0

    for step, (dense_x, sparse_x, label) in enumerate(train_loader): #step会循环len(train_loader)次，也就是总批次数
        # 1. 搬运数据到 GPU
        dense_x, sparse_x, label = dense_x.to(device), sparse_x.to(device), label.to(device)

        # 2. 梯度清零
        optimizer.zero_grad()

        # 3. 前向传播 (Forward)
        outputs = model(dense_x, sparse_x)
        
        # 4. 计算 Loss
        # 注意：outputs 是 [batch, 1]，label 是 [batch]，需要 view(-1, 1) 对齐维度 
        # view实际上就是reshape，-1代表自动识别应该要多少行或者列才能保证逗号前后相乘等于原来的总共元素数
        loss = criterion(outputs, label.view(-1, 1))

        # 5. 反向传播 (Backward)
        loss.backward()

        # 6. 更新参数 (Update)
        optimizer.step() 
        # loss.backward() 才是自动微分的核心（计算梯度）。
        #它从计算得到的标量损失值 loss 开始，沿着网络向前传播的路径逆向追溯。
        # optimizer.step() 只是负责“更新参数”（拿着算好的梯度去修改权重）。

        total_loss += loss.item()
        '''oss 是一个 PyTorch 张量，即使它只包含一个标量值。
        .item() 方法的作用是：将这个 PyTorch 张量中的标量值提取出来，
        转换为一个标准的 Python 浮点数。'''

        # 每 50 个 batch 打印一次进度
        if (step + 1) % 50 ==0:
            print(f"[Epoch {epoch_idx}] Step {step + 1} / {len(train_loader)} | Loss：{loss.item():.4f}")
            
    return total_loss / len(train_loader)

# === 3. 定义验证函数 (Evaluation Engine) ===   
def evaluate(model, val_loader):
    model.eval() # 切换到评估模式 (关闭 Dropout)
    all_targets = []
    all_preds = []

    with torch.no_grad(): # 验证时不需要计算梯度，省显存
        for dense_x, sparse_x, label in val_loader:
            dense_x, sparse_x, label = dense_x.to(device), sparse_x.to(device), label.to(device)

            outputs = model(dense_x, sparse_x)

            # 这里的 outputs 是 Logit，需要过 Sigmoid 变成概率 (0~1),推理阶段和训练阶段不同，不用更新参数，需要直观地利用sigmoid来反映概率，而不是没有直观概率意义的logit
            probs = torch.sigmoid(outputs).cpu().numpy() #numpy兼容cpu，而sklearn需要numpy格式为输入，这是python的东西
            targets = label.cpu().numpy()

            all_targets.extend(targets) #extend将可迭代对象的元素拆开来加入新的对象而不是append那种整个加入新对象
            all_preds.extend(probs)
            
    # 计算核心指标
    auc = roc_auc_score(all_targets, all_preds)
    loss = log_loss(all_targets, all_preds) #L_i = - [y_i \log(p_i) + (1 - y_i) \log(1 - p_i)]
    return auc, loss

# === 4. 主循环 (Main Loop) ===
EPOCHS = 15 # 训练 15 轮

print(f"Start Training for {EPOCHS} epochs...")
start_time = time.time() #返回当前的系统时间戳

for epoch in range(EPOCHS):
    print(f"\n--- Epoch {epoch + 1} ---")

    # 1. 训练一轮
    train_loss = train_one_epoch(model, train_loader, epoch + 1)

    # 2. 验证效果
    auc, val_loss = evaluate(model, val_loader)

    print(f"Epoch {epoch + 1} Result:")
    print(f"  Train Loss: {train_loss:.4f}")
    print(f"  Val Loss:   {val_loss:.4f}")
    print(f"  Val AUC:    {auc:.4f}")

print(f"\nTraining Done! Total Time: {time.time() - start_time}s")
    



Using device: cuda
Start Training for 15 epochs...

--- Epoch 1 ---
[Epoch 1] Step 50 / 3125 | Loss：15.4479
[Epoch 1] Step 100 / 3125 | Loss：11.6486
[Epoch 1] Step 150 / 3125 | Loss：11.8771
[Epoch 1] Step 200 / 3125 | Loss：9.5397
[Epoch 1] Step 250 / 3125 | Loss：10.7418
[Epoch 1] Step 300 / 3125 | Loss：9.0293
[Epoch 1] Step 350 / 3125 | Loss：8.4517
[Epoch 1] Step 400 / 3125 | Loss：7.6424
[Epoch 1] Step 450 / 3125 | Loss：7.6633
[Epoch 1] Step 500 / 3125 | Loss：7.2922
[Epoch 1] Step 550 / 3125 | Loss：7.2655
[Epoch 1] Step 600 / 3125 | Loss：5.5817
[Epoch 1] Step 650 / 3125 | Loss：6.2814
[Epoch 1] Step 700 / 3125 | Loss：4.5609
[Epoch 1] Step 750 / 3125 | Loss：4.7925
[Epoch 1] Step 800 / 3125 | Loss：5.2714
[Epoch 1] Step 850 / 3125 | Loss：4.3837
[Epoch 1] Step 900 / 3125 | Loss：4.8590
[Epoch 1] Step 950 / 3125 | Loss：4.5004
[Epoch 1] Step 1000 / 3125 | Loss：4.5030
[Epoch 1] Step 1050 / 3125 | Loss：3.9964
[Epoch 1] Step 1100 / 3125 | Loss：4.1833
[Epoch 1] Step 1150 / 3125 | Loss：4.2730
[Epoc